# a decorator to set the log level of a function


In [2]:
from loguru import logger as lg

In [3]:
from functools import wraps

### Basic


In [4]:
def log_leveller(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        lg.debug(
            f"Function {func.__name__} called with args: {args} and kwargs: {kwargs}",
        )
        return func(*args, **kwargs)

    return wrapper


@log_leveller
def sample(a: int) -> int:
    lg.info(f"Function sample called with arg: {a}")
    b = a + 1
    lg.debug(f"Function sample returning: {b}")
    return b


sample(1)

2024-06-11 00:21:16.708 | DEBUG    | __main__:wrapper:4 - Function sample called with args: (1,) and kwargs: {}
2024-06-11 00:21:16.709 | INFO     | __main__:sample:14 - Function sample called with arg: 1
2024-06-11 00:21:16.710 | DEBUG    | __main__:sample:16 - Function sample returning: 2


2

### With params


- https://realpython.com/primer-on-python-decorators/#defining-decorators-with-arguments


In [5]:
def repeat(num_times):
    def decorator_repeat(func):
        @wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value

        return wrapper_repeat

    return decorator_repeat


@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")


greet("World")

Hello World
Hello World
Hello World
Hello World


In [6]:
import os
import sys


def log_leveller_with_args(level):
    def decorator_log_leveller(func):

        @wraps(func)
        def wrapper(*args, **kwargs):
            # save the current log level
            current_level = lg.level("SUCCESS")
            lg.warning(f"Current log level lg: {current_level}")

            # current_level = os.environ.get("LOGURU_LEVEL", "INFO")
            # lg.warning(f"Current log level os: {current_level}")

            # # set the log level
            lg.warning(f"Set log level to: {level}")
            # os.environ["LOGURU_LEVEL"] = level
            lg.remove()
            lg.add(sys.stderr, level=level.upper())

            lg.warning(f"Func {func.__name__} called with {args=} and {kwargs=}")
            ret = func(*args, **kwargs)

            # # reset the log level
            lg.warning(f"Reset log level to: {current_level}")
            # os.environ["LOGURU_LEVEL"] = current_level
            lg.remove()
            lg.add(sys.stderr, level=current_level.name.upper())

            return ret

        return wrapper

    return decorator_log_leveller


@log_leveller_with_args(level="DEBUG")
# @log_leveller_with_args(level="INFO")
# @log_leveller_with_args(level="WARNING")
def sample(a: int) -> int:
    lg.info(f"Function sample called with arg: {a}")
    b = a + 1
    lg.debug(f"Function sample returning: {b}")
    return b


def standard(a=1):
    lg.success(f"Starting standard with {a=}")
    lg.info(f"Function standard called with arg: {a}")
    b = a + 1
    lg.debug(f"Function standard returning: {b}")
    return b


standard(1)
sample(1)
standard(1)
sample(1)
standard(1)

2024-06-11 00:21:16.736 | SUCCESS  | __main__:standard:50 - Starting standard with a=1
2024-06-11 00:21:16.737 | INFO     | __main__:standard:51 - Function standard called with arg: 1
2024-06-11 00:21:16.738 | DEBUG    | __main__:standard:53 - Function standard returning: 2
2024-06-11 00:21:16.739 | WARNING  | __main__:wrapper:12 - Current log level lg: Level(name='SUCCESS', no=25, color='<green><bold>', icon='✅')
2024-06-11 00:21:16.740 | WARNING  | __main__:wrapper:18 - Set log level to: DEBUG
2024-06-11 00:21:16.748 | WARNING  | __main__:wrapper:23 - Func sample called with args=(1,) and kwargs={}
2024-06-11 00:21:16.748 | INFO     | __main__:sample:43 - Function sample called with arg: 1
2024-06-11 00:21:16.749 | DEBUG    | __main__:sample:45 - Function sample returning: 2
2024-06-11 00:21:16.750 | WARNING  | __main__:wrapper:27 - Reset log level to: Level(name='SUCCESS', no=25, color='<green><bold>', icon='✅')
2024-06-11 00:21:16.759 | SUCCESS  | __main__:standard:50 - Starting st

2

In [7]:
# lg._core.handlers
# {28: (id=28, level=25, sink=stderr)}
# list(lg._core.handlers.items())[0][1]._sink

In [16]:
from mod import standard_mod, another_mod

# from mod import standard_mod

lg.warning("Starting")
standard(1)
standard_mod(1)
another_mod(1)

lg.warning("Disabling standard_mod")
# lg.disable("__main__:standard")
lg.disable("mod")
# lg.disable("mod.standard_mod")
standard(1)
standard_mod(1)
another_mod(1)

lg.warning("Enabling standard_mod")
# lg.enable("__main__:standard")
lg.enable("mod")
# lg.enable("mod.standard_mod")
standard(1)
standard_mod(1)
another_mod(1)

2024-06-11 00:23:56.527 | WARNING  | __main__:<module>:5 - Starting
2024-06-11 00:23:56.528 | SUCCESS  | __main__:standard:50 - Starting standard with a=1
2024-06-11 00:23:56.530 | SUCCESS  | mod:standard_mod:9 - Starting standard with a=1
2024-06-11 00:23:56.530 | SUCCESS  | mod:another_mod:17 - Starting another with a=1
2024-06-11 00:23:56.531 | WARNING  | __main__:<module>:10 - Disabling standard_mod
2024-06-11 00:23:56.531 | SUCCESS  | __main__:standard:50 - Starting standard with a=1
2024-06-11 00:23:56.532 | WARNING  | __main__:<module>:18 - Enabling standard_mod
2024-06-11 00:23:56.533 | SUCCESS  | __main__:standard:50 - Starting standard with a=1
2024-06-11 00:23:56.534 | SUCCESS  | mod:standard_mod:9 - Starting standard with a=1
2024-06-11 00:23:56.534 | SUCCESS  | mod:another_mod:17 - Starting another with a=1


2

In [9]:
# from mod import standard_mod
from mod import standard_mod, another_mod

# from mod import test